# Entregable 2 de CoderHouse:
API: https://api-docs.luchtmeetnet.nl/#6ac38ef4-5d43-4d9b-b0b9-69b2d00e6f4c

In [ ]:
%pip install sqlalchemy
%pip install psycopg2-binary
%pip install requests


In [ ]:
url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws"
port="5439"
data_base="data-engineer-database"
user="hgr79_coderhouse"
pwd="kye79Z96x3Xe"

# with open("/Users/hernangonzalez/Downloads/pwd_coder.txt",'r') as f:
#     pwd= f.read()

In [ ]:
from sqlalchemy import create_engine
import psycopg2.extensions
psycopg2.extensions.register_type(psycopg2.extensions.UNICODE)
import pandas as pd

#conn = create_engine(f'postgresql://{user}:{pwd}@{url}.com:5439/{data_base}')
# conn = create_engine(url=url, user=user, pwd=pwd, port=port, database=data_base)

In [ ]:
try:
    conn = psycopg2.connect(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
        dbname=data_base,
        user=user,
        password=pwd,
        port='5439'
    )
    print("Connected to Redshift successfully!")
    
    # Do some work with the connection here...
    
except Exception as e:
    print("Unable to connect to Redshift.")
    print(e)

In [ ]:
conn

In [ ]:
import requests

headers = {"Accept-Encoding": "gzip, deflate"}
#response = requests.get("https://api.luchtmeetnet.nl/open_api/measurements?start=2018-12-01T09%3A00%3A00Z&end=2018-12-02T09%3A00%3A00Z&station_number=&formula=&page=&order_by=timestamp_measured&order_direction=desc")
response = requests.get("https://api.luchtmeetnet.nl/open_api/measurements?station_number=&formula=&page=1&order_by=timestamp_measured&order_direction=desc",headers=headers)
print(response)
file_name = response.json()
file_name

In [ ]:
print(type(file_name))
datax=file_name['data']
df = pd.DataFrame.from_dict(datax)
df

In [ ]:
df.drop(df.index[50:1000], inplace=True)
#df.drop(columns=['timestamp_measured'])

In [ ]:

df.insert(0,'id','')
df['id']=df.index+1
#df=df.reset_index(drop=False)
#df.rename(columns = {'index':'id'}, inplace = True)
df["timestamp_measured"] = pd.to_datetime(df["timestamp_measured"])
df["station_number"] = df["station_number"].astype("string")
df["formula"] = df["formula"].astype("string")

print(df.shape)
print(df.dtypes)
print(df.info())
df.describe


In [ ]:
cur = conn.cursor()
# Execute a SQL query to select data from a table
cur.execute("SELECT * FROM aq")
# Fetch the results
results = cur.fetchall()
results

In [ ]:
from psycopg2.extras import execute_values
cur = conn.cursor()

table_name = 'aq'
# Define the columns you want to insert data into
columns = ['id', 'station_number', 'value', 'timestamp_measured','formula']
# Convert the DataFrame to a list of tuples
values = [tuple(x) for x in df.to_numpy()]
insert_sql = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES %s"
# Execute the INSERT statement using execute_values
cur.execute("BEGIN")
execute_values(cur, insert_sql, values)
cur.execute("COMMIT")
# Close the cursor and connection
cur.close()
conn.close()